In [2]:
import numpy as np
import pandas as pd
import os

import vodex as vx
import numan as nu
import tifffile as tif
from patchify import patchify, unpatchify

from typing import Union, List, Optional, Tuple, Dict, Any
import numpy.typing as npt

In [3]:
test_movie = np.zeros((20,10,10,10), dtype=np.int16)
test_movie[:,0:2,3:5,2:4]=1
test_movie[:,0:2,3:5,4:6]=10
test_movie = test_movie*np.arange(20)[:,None,None,None]
#print(test_movie)
tif.imwrite('test_movie.tif', test_movie.astype(np.int16), shape=(20,10,10,10), 
            metadata={'axes': 'TZYX'}, imagej=True)

In [4]:
test_data = np.zeros((10,10,10), dtype=np.int16)
test_data[0:5,0:5,0:5]=1
test_data[5:,5:,5:]=2
tif.imwrite('test_mask.tif', test_data, imagej=True)

In [101]:
class Voxelizer:
    
    """
    Compute average signal value (of a 4D matrix of fluorescence imaging data: fluorescence intensity at t X z X y X x) across Super Voxels (3D voxels with user-defined size)
    inside a ROI (user-defined mask -multiple ROIs can be provided by providing different integer IDs into the mask).
    Note: the mask should have the same 3D dimentions as a dataset volume and consist of integer values corresponding to voxels of interest 
    for the different ROIs; the 3D Super Voxel size should be no bigger than the dataset volume. 

    Args:
        mask_file: strig of a .tif file with the same 3D dimentions of the dataset volume and integer values correspondent to the voxels of the different ROIs
        superVoxel_size: a 1x3 array [z,y,x] with the dimentions of the Super Voxel (group of voxels to be averaged and considered as single unit)
        roi_id: an integer value correpsondent to the ROI to be analysed (to be patchified in SVs and compute its SV avg); this value is the 
            integer value of the voxels in mask_file correpsondent to the ROI of interest

    Attributes:
        SV_size: a 1x3 array [z,y,x] with the dimentions of the Super Voxel (group of voxels to be averaged and considered as single unit)
        mask: an numpy array with the same 3D dimentions of the dataset volume with integer values correspondent to the voxels of the different ROIs
        padded_mask: as mask but with dimentions zero-padded to fit a integer number of SVs
        patched_mask: as padded_mask but structured in patches of SV dimentions
        mask_SV: list of mask patches values correpsondent to the only ROI of interest
        x_min,x_max,y_min,y_max,z_min,z_max: lists with the coordinates of mask_SV (corresponding to the elemnts in mask_SV)
    """
    
    def __init__(self, mask_file: str, superVoxel_size: npt.NDArray, roi_id: int):
        self.SV_size = superVoxel_size # order: z,y,x
        # define list with SV coordinates
        self.mask_SV = []
        self.x_min = [] 
        self.x_max = [] 
        self.y_min = [] 
        self.y_max = [] 
        self.z_min = []
        self.z_max = []
        # load multi-ROI mask from .tif
        self.mask = self._load_mask(mask_file)
        # zero pad the mask to fit an integer number of SVs
        self.padded_mask = self._zero_pad_mask()
        # patchify mask through SV
        self.patched_mask = self._patchify_mask()
        # extract patches related to a specific ROI
        self._get_ROI_patches(roi_id)

        
    def process_movie(self, experiment: vx.Experiment, batch_size: int) -> npt.NDArray: #patchify in SV and calculated avg signal inside each SV
        
        """
        Calculate mean signal values inside SV within the ROI

        Args:
            experiment: Vodex Experiment object
            batch_size: number of movie volumes (time points) to be loaded and process at a time

        Returns:
            voxel_table: table of all masked SV (in rows) means across time (in columns). 
            Note: the first 6 columns [0:5] aren't reporting avg signal but the x_min, x_max, y_min, y_max, z_min, z_max coordinates of the relative SV
        """
        
        self.SV_avg_matrix = np.zeros((len(self.mask_SV), experiment.n_volumes+6))
        print(self.SV_avg_matrix.shape)
        chuncks = experiment.batch_volumes(batch_size, full_only=True, overlap=0)
        for i_patch in np.arange(len(self.mask_SV)):
            self.SV_avg_matrix[i_patch, 0] = self.x_min[i_patch]
            self.SV_avg_matrix[i_patch, 1] = self.x_max[i_patch]
            self.SV_avg_matrix[i_patch, 2] = self.y_min[i_patch]
            self.SV_avg_matrix[i_patch, 3] = self.y_max[i_patch]
            self.SV_avg_matrix[i_patch, 4] = self.z_min[i_patch]
            self.SV_avg_matrix[i_patch, 5] = self.z_max[i_patch]
            vol_count = 0
            for chunck in chuncks:
                data = experiment.load_volumes(chunck, verbose=False)
                # cut dataset around SV
                a = data[:, self.z_min[i_patch]:self.z_max[i_patch], self.y_min[i_patch]:self.y_max[i_patch], self.x_min[i_patch]:self.x_max[i_patch]]
                #print(a.shape)
                # apply mask to SV
                b = a*self.mask_SV[i_patch]
                #print(b.shape)
                # avg inside SV
                c = np.sum(a, axis=(1,2,3))/np.sum(self.mask_SV[i_patch])
                #print(c.shape)
                self.SV_avg_matrix[i_patch,(vol_count+6):(vol_count+6+batch_size)] = c.T
                vol_count += batch_size
                
                                                                                                      
    def _load_mask(self, mask_file) -> npt.NDArray: 
        mask = tif.imread(mask_file)
        return mask
    
    def _zero_pad_mask(self) -> npt.NDArray: 
        z_pad = (self.mask.shape[0]%self.SV_size[0], self.mask.shape[0]%self.SV_size[0])
        y_pad = (self.mask.shape[1]%self.SV_size[1], self.mask.shape[1]%self.SV_size[1])
        x_pad = (self.mask.shape[2]%self.SV_size[2], self.mask.shape[2]%self.SV_size[2])
        padded = np.pad(self.mask, pad_width=[z_pad, y_pad, x_pad], mode='constant')
        return padded
    
    def _patchify_mask(self) -> npt.NDArray:
        patched = patchify(self.padded_mask, self.SV_size, step=self.SV_size)
        return patched
    
    def _get_ROI_patches(self, roi_id: int):
        self.mask_SV = []
        self.x_min = [] 
        self.x_max = [] 
        self.y_min = [] 
        self.y_max = [] 
        self.z_min = []
        self.z_max = []
        for i_SV_z in np.arange(self.patched_mask.shape[0]):
            for i_SV_y in np.arange(self.patched_mask.shape[1]):
                for i_SV_x in np.arange(self.patched_mask.shape[2]):
                    my_patch = self.patched_mask[i_SV_z, i_SV_y, i_SV_x, :, :, :] == roi_id
                    #print('patch')
                    if np.sum(my_patch) > 0:
                        #print(my_patch)
                        self.mask_SV.append(my_patch)
                        self.x_min.append(i_SV_x*self.SV_size[2])
                        self.x_max.append((i_SV_x+1)*self.SV_size[2]) #+1 ?
                        self.y_min.append(i_SV_y*self.SV_size[1])
                        self.y_max.append((i_SV_y+1)*self.SV_size[1]) #+1 ?
                        self.z_min.append(i_SV_z*self.SV_size[0])
                        self.z_max.append((i_SV_z+1)*self.SV_size[0]) #+1 ?

# Test

In [102]:
movie_dir = "test_movie"
mask_file = "test_mask.tif"
superVoxel_size = [3,3,3] # order: z,y,x
roi_id = 1 #select mask, the test checks fit mask 1, not 2
n_vol_batch = 3

frames_per_volume = 10
starting_slice = 0
experiment = vx.Experiment.from_dir(movie_dir, frames_per_volume, starting_slice, verbose=False)

voxelizer = Voxelizer(mask_file, superVoxel_size, roi_id)
#voxelizer.process_movie(experiment, n_vol_batch)
#final_cell_matrix = voxelizer.create_cell_signal_table(experiment)

In [25]:
mask = voxelizer._load_mask(mask_file)
print(mask.shape)
np.sum(mask) == 5*5*5*1 + 5*5*5*2

(10, 10, 10)


True

In [29]:
padded_mask = voxelizer._zero_pad_mask()
print(padded_mask.shape)
# this works for SV of size [3,3,3]
padded_mask.shape == (12,12,12)

(12, 12, 12)


True

In [52]:
patched_mask = voxelizer._patchify_mask()
print(patched_mask.shape)
# this works for SV of size [3,3,3]
patched_mask.shape == (4,4,4,3,3,3)

(4, 4, 4, 3, 3, 3)


True

In [63]:
voxelizer._get_ROI_patches(1)
len(voxelizer.mask_SV) == 8

True

In [108]:
voxelizer.process_movie(experiment, 2)

final_cell_df =  pd.DataFrame(voxelizer.SV_avg_matrix)
print(final_cell_df)

(8, 26)
    0    1    2    3    4    5    6         7         8     9   ...  \
0  0.0  3.0  0.0  3.0  0.0  3.0  0.0  0.000000  0.000000   0.0  ...   
1  3.0  6.0  0.0  3.0  0.0  3.0  0.0  0.000000  0.000000   0.0  ...   
2  0.0  3.0  3.0  6.0  0.0  3.0  0.0  0.333333  0.666667   1.0  ...   
3  3.0  6.0  3.0  6.0  0.0  3.0  0.0  4.666667  9.333333  14.0  ...   
4  0.0  3.0  0.0  3.0  3.0  6.0  0.0  0.000000  0.000000   0.0  ...   
5  3.0  6.0  0.0  3.0  3.0  6.0  0.0  0.000000  0.000000   0.0  ...   
6  0.0  3.0  3.0  6.0  3.0  6.0  0.0  0.000000  0.000000   0.0  ...   
7  3.0  6.0  3.0  6.0  3.0  6.0  0.0  0.000000  0.000000   0.0  ...   

          16         17    18         19         20    21         22  \
0   0.000000   0.000000   0.0   0.000000   0.000000   0.0   0.000000   
1   0.000000   0.000000   0.0   0.000000   0.000000   0.0   0.000000   
2   3.333333   3.666667   4.0   4.333333   4.666667   5.0   5.333333   
3  46.666667  51.333333  56.0  60.666667  65.333333  70.0  74.66